Here we remap our hippocampal labels to contain 4 layers in hipp, and 2 layers in DG. This is so that even if there are 0 voxels separating a fold, a midthickness surface can still be generated that doesn't take a "bridge" shortcut. 
We also remap background labels to keep them simpler.
Next we register hippocampi to background label hippocampi and superimpose them. 

In [20]:
import numpy as np
import nibabel as nib

In [21]:
cmd1 = "ls bgs/*_preproc.nii.gz"
bgs = !{cmd1}
bgs

['bgs/bg-0-aparcaseg_preproc.nii.gz',
 'bgs/bg-1-aparcaseg_preproc.nii.gz',
 'bgs/bg-2-aparcaseg_preproc.nii.gz']

In [22]:
hippunfold_dir = '../BIDS_multihist7/hippunfold_v2.0.0beta_corobl/'
cmd1 = f"ls ../BIDS_multihist7/corobl/sub-*/anat/sub-*desc-tissue_dseg.nii.gz"
fgs = !{cmd1}
print(fgs[0].split('/')[-1].split('_space')[0])
fgs

sub-122017_hemi-L_desc-tissue_dseg.nii.gz


['../BIDS_multihist7/corobl/sub-122017/anat/sub-122017_hemi-L_desc-tissue_dseg.nii.gz',
 '../BIDS_multihist7/corobl/sub-122017/anat/sub-122017_hemi-R_desc-tissue_dseg.nii.gz',
 '../BIDS_multihist7/corobl/sub-152017/anat/sub-152017_hemi-L_desc-tissue_dseg.nii.gz',
 '../BIDS_multihist7/corobl/sub-152017/anat/sub-152017_hemi-R_desc-tissue_dseg.nii.gz',
 '../BIDS_multihist7/corobl/sub-bbhist/anat/sub-bbhist_hemi-L_desc-tissue_dseg.nii.gz',
 '../BIDS_multihist7/corobl/sub-bbhist/anat/sub-bbhist_hemi-R_desc-tissue_dseg.nii.gz',
 '../BIDS_multihist7/corobl/sub-pli3d/anat/sub-pli3d_hemi-L_desc-tissue_dseg.nii.gz']

In [23]:
# new labeling scheme that keeps only bg, csf, gm, wm, blood, choroid
# note labels 1-4 are hipp gm layers, 5-6 are dg layers
remap_HU = {#1-4 gm layers
    #5-6 dg layers
    2:7, #SRLM
    7:8, #cyst
    5:9, #HATA
    6:10, #ind.gris
    9:11, #dgsrc
    10:12, #dgsink
    11:13} #alveus
remap_FS = {0:0,
    2:18, #wm
    4:0, #ventricle
    5:0, #ventricle
    7:18, #cerebellar wm
    8:14, #cerebellar gm
    10:14, #thalamus
    11:14, #caudate
    12:14, #putamen
    13:14, #pallidum
    16:14, #brain stem
    17:0, #hippocampus. we will treat this as csf to fill in missing ventricle areas
    18:14, #amygdala
    24:0, #ventricle
    28:14, #centralDC
    31:17, # choroid plexus
    77:18, # wm hypointensity
    41:18, #wm
    43:0, #ventricle
    44:0, #ventricle
    46:18, #cerebellar wm
    47:14, #cerebellar gm
    49:14, #thalamus
    50:14, #caudate
    51:14, #putamen
    52:14, #pallidum
    53:0, #hippocampus. we will treat this as csf to fill in missing ventricle areas
    54:14, #amygdala
    60:14, #centralDC
    63:17, # choroid plexus
    77:18, # wm hypointensity
    1000:15, # neocortex
    101:16} #blood


In [24]:
!mkdir -p labelmaps
!mkdir -p tmp
!rm -r tmp/*
!rm -r labelmaps/*.nii.gz

cmdprefix = f"singularity exec /data/mica1/01_programs/singularity/hippunfold_deps.sif "
i=0
for fgorig in fgs:

    sub, hemi = fgorig.split("/")[-1].split(".")[0].split("_")[:2]
    cmd = f"{cmdprefix} antsApplyTransforms -d 3 -i {fgorig} -r ref-corobl_200um.nii.gz -o tmp/orig.nii.gz -n MultiLabel"
    !{cmd}

    # combine L-L, R-R, L-R, and R-L
    cmd0 = f"{cmdprefix} c3d tmp/orig.nii.gz -flip x -o tmp/flip.nii.gz"
    !{cmd0}
    if hemi == "hemi-L":
        hipplbls = [17, 53]
    elif hemi == "hemi-R":
        hipplbls = [53, 17]
    else:
        raise ValueError("Invalid hemisphere label")

    for f,fg in enumerate([f"tmp/orig.nii.gz",f"tmp/flip.nii.gz"]):

        # get coords
        coords_hipp = f"{hippunfold_dir}/{sub}/coords/{sub}_dir-IO_{hemi}_space-corobl_label-hipp_desc-equidist_coords.nii.gz"
        cmd3 = f"{cmdprefix} antsApplyTransforms -d 3 -i {coords_hipp} -r ref-corobl_200um.nii.gz -o tmp/coords_hipp.nii.gz -n NearestNeighbor"
        !{cmd3}
        coords_dg = f"{hippunfold_dir}/{sub}/coords/{sub}_dir-IO_{hemi}_space-corobl_label-dentate_desc-equidist_coords.nii.gz"
        cmd4 = f"{cmdprefix} antsApplyTransforms -d 3 -i {coords_dg} -r ref-corobl_200um.nii.gz -o tmp/coords_dentate.nii.gz -n NearestNeighbor"
        !{cmd4}
        if f==1:
            cmd1 = f"{cmdprefix} c3d tmp/coords_hipp.nii.gz -flip x -o tmp/coords_hipp_flip.nii.gz"
            !{cmd1}
            cmd2 = f"{cmdprefix} c3d tmp/coords_dentate.nii.gz -flip x -o tmp/coords_dentate_flip.nii.gz"
            !{cmd2}
            flip = "_flip"
        else:
            flip= ""
        coords_hipp_dat = nib.load(f"tmp/coords_hipp{flip}.nii.gz").get_fdata()
        coords_dentate_dat = nib.load(f"tmp/coords_dentate{flip}.nii.gz").get_fdata()
        fg_nii = nib.load(fg)
        fg_dat = fg_nii.get_fdata()

        hipplbl = hipplbls[f]
        for b,bg in enumerate(bgs):
            
            # align fg and bg
            cmd1 = f"{cmdprefix} c3d {bg} -retain-labels {hipplbl} -binarize -smooth 0.5mm -o tmp/bgbin.nii.gz"
            !{cmd1}
            cmd2 = f"{cmdprefix} c3d {fg} -retain-labels 1 2 8 11 -binarize -smooth 0.5mm -o tmp/fgbin.nii.gz"
            !{cmd2}
            cmd3 = f"{cmdprefix} greedy -threads 32 -d 3 -m SSD -moments 1 -det 1 -i tmp/fgbin.nii.gz tmp/bgbin.nii.gz -o tmp/xfm.txt"
            !{cmd3}
            cmd4 = f"{cmdprefix} greedy -threads 32 -d 3 -m SSD -s 1.732vox 0.7071vox -i tmp/fgbin.nii.gz tmp/bgbin.nii.gz -it tmp/xfm.txt -o tmp/xfm.nii.gz"
            !{cmd4}
            cmd5 = f"{cmdprefix} greedy -d 3 -threads 32 -ri LABEL 0.2vox -rf tmp/fgbin.nii.gz -rm {bg} tmp/bg_space-fg.nii.gz -r tmp/xfm.nii.gz tmp/xfm.txt"
            !{cmd5}

            # combine labels
            bg_dat = np.round(nib.load(f"tmp/bg_space-fg.nii.gz").get_fdata())
            bg_dat[bg_dat>1000] = 1000

            newimg = np.zeros_like(fg_dat)
            for old, new in remap_FS.items():
                newimg[bg_dat==old] = new
            for old, new in remap_HU.items():
                newimg[fg_dat==old] = new
            newimg[fg_dat==1] = np.floor(coords_hipp_dat[fg_dat==1]*4) +1
            newimg[fg_dat==8] = np.floor(coords_dentate_dat[fg_dat==8]*2) +5
            nib.save(nib.Nifti1Image(newimg.astype(int),affine=fg_nii.affine,header=fg_nii.header),f"labelmaps/{sub}_{hemi}_bg-{b}_{flip}.nii.gz")
            i+=1

        # make some with no bg
        newimg = np.zeros_like(fg_dat)
        for old, new in remap_HU.items():
            newimg[fg_dat==old] = new
        newimg[fg_dat==1] = np.floor(coords_hipp_dat[fg_dat==1]*4) +1
        newimg[fg_dat==8] = np.floor(coords_dentate_dat[fg_dat==8]*2) +5

        nib.save(nib.Nifti1Image(newimg.astype(int),affine=fg_nii.affine,header=fg_nii.header),f"labelmaps/{sub}_{hemi}_bg-none_{flip}.nii.gz")
        i+=1


!rm -r tmp/*

rm: cannot remove 'tmp/*': No such file or directory
Limiting the number of threads to 32
--- MATCHING BY MOMENTS OF ORDER 1 ---
Fixed Mean        : -24.0384 -14.4716 -10.5207
Fixed Covariance  : 
29.5731 -30.4898 13.8978
-30.4898 106.032 -22.5321
13.8978 -22.5321 16.9659

Moving Mean       : -21.5506 -13.7333 -15.8425
Moving Covariance : 
28.3544 -15.381 10.4189
-15.381 86.0809 -46.0447
10.4189 -46.0447 33.5882

Metric for flip 1 1 1 : 0.0252953
Limiting the number of threads to 32
LEVEL 1 of 4
  Smoothing sigmas: [2.7712, 2.7712, 2.7712], [1.13136, 1.13136, 1.13136]
Level 000  Iter 00000    Energy = 0.018881
Level 000  Iter 00001    Energy = 0.012515
Level 000  Iter 00002    Energy = 0.008689
Level 000  Iter 00003    Energy = 0.006420
Level 000  Iter 00004    Energy = 0.004738
Level 000  Iter 00005    Energy = 0.003646
Level 000  Iter 00006    Energy = 0.002873
Level 000  Iter 00007    Energy = 0.002332
Level 000  Iter 00008    Energy = 0.001931
Level 000  Iter 00009    Energy = 0.00

In [25]:
!rm -r tmp/*

rm: cannot remove 'tmp/*': No such file or directory
